In [1]:
pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 889.2 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from string import digits
import nltk
import re
import string
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
lines = pd.read_csv('/kaggle/input/hindi-english-parallel-corpus/hindi_english_parallel.csv')
lines = lines[:30000]
lines.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है,A list of plugins that are disabled by default


In [4]:
# Lowercase all characters
lines['english']=lines['english'].apply(lambda x: str(x))
lines['hindi']=lines['hindi'].apply(lambda x: str(x))
lines['english']=lines['english'].apply(lambda x: x.lower())
lines['hindi']=lines['hindi'].apply(lambda x: x.lower())

In [5]:
lines['hindi'][0]

'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'

In [6]:
# Remove quotes
lines['english']=lines['english'].apply(lambda x: re.sub("'", '', x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub("'", '', x))

In [7]:
lines.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,the default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है,a list of plugins that are disabled by default


In [8]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [9]:
lines.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका,the default plugin layout for the top panel
4,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है,a list of plugins that are disabled by default


In [10]:
remove_digits = str.maketrans('', '', digits)

In [11]:
remove_digits

{48: None,
 49: None,
 50: None,
 51: None,
 52: None,
 53: None,
 54: None,
 55: None,
 56: None,
 57: None}

In [12]:
a = lines['english'][0].translate(remove_digits)

In [13]:
a

'give your application an accessibility workout'

In [14]:
a.strip()

'give your application an accessibility workout'

In [15]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english']=lines['english'].apply(lambda x: x.translate(remove_digits))
lines['hindi']=lines['hindi'].apply(lambda x: x.translate(remove_digits))

lines['hindi'] = lines['hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english']=lines['english'].apply(lambda x: x.strip())
lines['hindi']=lines['hindi'].apply(lambda x: x.strip())
lines['english']=lines['english'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi']=lines['hindi'].apply(lambda x: re.sub(" +", " ", x))

In [16]:
'hello! how are you buddy?'.strip()

'hello! how are you buddy?'

In [17]:
lines['english'][0]

'give your application an accessibility workout'

In [18]:
# Add start and end tokens to target sequences
lines['hindi'] = lines['hindi'].apply(lambda x : 'START_ '+ x + ' _END')

In [19]:
lines['hindi'][0]

'START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें _END'

In [20]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [21]:
lines.head()

,hindi,english
0,START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें _END,give your application an accessibility workout
1,START_ एक्सेर्साइसर पहुंचनीयता अन्वेषक _END,accerciser accessibility explorer
2,START_ निचले पटल के लिए डिफोल्ट प्लगइन खाका _END,the default plugin layout for the bottom panel
3,START_ ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका _END,the default plugin layout for the top panel
4,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है _END,a list of plugins that are disabled by default


In [22]:
lines['length_eng']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_hin']=lines['hindi'].apply(lambda x:len(x.split(" ")))

In [23]:
lines.head()
lines[lines['length_eng']>30].shape

(102, 4)

In [24]:
lines=lines[lines['length_eng']<=20]
lines=lines[lines['length_hin']<=20]

In [25]:
print("maximum length of Hindi Sentence ",max(lines['length_hin']))
print("maximum length of English Sentence ",max(lines['length_eng']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [26]:
max_length_src=max(lines['length_hin'])
max_length_tar=max(lines['length_eng'])

In [27]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(2688, 3438)

In [28]:
num_decoder_tokens

3438

In [29]:
num_decoder_tokens += 1

In [30]:
num_decoder_tokens

3439

In [31]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [32]:
input_token_index

{'a': 1,
 'able': 2,
 'abort': 3,
 'aborted': 4,
 'about': 5,
 'above': 6,
 'absolute': 7,
 'ac': 8,
 'accelerator': 9,
 'accelerator…': 10,
 'accentuated': 11,
 'accept': 12,
 'acceptable': 13,
 'accepted': 14,
 'accerciser': 15,
 'access': 16,
 'accessed': 17,
 'accessibility': 18,
 'accessible': 19,
 'accessibles': 20,
 'according': 21,
 'accordion': 22,
 'ace': 23,
 'aces': 24,
 'across': 25,
 'act': 26,
 'acti': 27,
 'action': 28,
 'actionable': 29,
 'actions': 30,
 'activatable': 31,
 'activate': 32,
 'activated': 33,
 'active': 34,
 'actual': 35,
 'adaptive': 36,
 'add': 37,
 'added': 38,
 'addition': 39,
 'additional': 40,
 'addrcheck': 41,
 'address': 42,
 'addressbook': 43,
 'addresses': 44,
 'adds': 45,
 'add…': 46,
 'advance': 47,
 'advanced': 48,
 'advised': 49,
 'affects': 50,
 'after': 51,
 'again': 52,
 'against': 53,
 'agnes': 54,
 'aim': 55,
 'aisleriot': 56,
 'al': 57,
 'albert': 58,
 'alert': 59,
 'algorithm': 60,
 'alidate': 61,
 'all': 62,
 'alleys': 63,
 'allocat

In [33]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [34]:
reverse_input_char_index

{1: 'a',
 2: 'able',
 3: 'abort',
 4: 'aborted',
 5: 'about',
 6: 'above',
 7: 'absolute',
 8: 'ac',
 9: 'accelerator',
 10: 'accelerator…',
 11: 'accentuated',
 12: 'accept',
 13: 'acceptable',
 14: 'accepted',
 15: 'accerciser',
 16: 'access',
 17: 'accessed',
 18: 'accessibility',
 19: 'accessible',
 20: 'accessibles',
 21: 'according',
 22: 'accordion',
 23: 'ace',
 24: 'aces',
 25: 'across',
 26: 'act',
 27: 'acti',
 28: 'action',
 29: 'actionable',
 30: 'actions',
 31: 'activatable',
 32: 'activate',
 33: 'activated',
 34: 'active',
 35: 'actual',
 36: 'adaptive',
 37: 'add',
 38: 'added',
 39: 'addition',
 40: 'additional',
 41: 'addrcheck',
 42: 'address',
 43: 'addressbook',
 44: 'addresses',
 45: 'adds',
 46: 'add…',
 47: 'advance',
 48: 'advanced',
 49: 'advised',
 50: 'affects',
 51: 'after',
 52: 'again',
 53: 'against',
 54: 'agnes',
 55: 'aim',
 56: 'aisleriot',
 57: 'al',
 58: 'albert',
 59: 'alert',
 60: 'algorithm',
 61: 'alidate',
 62: 'all',
 63: 'alleys',
 64: 'all

In [35]:
lines.head(10)

,hindi,english,length_eng,length_hin
0,START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें _END,give your application an accessibility workout,6,10
1,START_ एक्सेर्साइसर पहुंचनीयता अन्वेषक _END,accerciser accessibility explorer,3,5
2,START_ निचले पटल के लिए डिफोल्ट प्लगइन खाका _END,the default plugin layout for the bottom panel,8,9
3,START_ ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका _END,the default plugin layout for the top panel,8,9
4,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है _END,a list of plugins that are disabled by default,9,14
5,START_ अवधि को हाइलाइट रकें _END,highlight duration,2,6
6,START_ पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि _END,the duration of the highlight box when selecting accessible nodes,10,12
7,START_ सीमांत बोर्डर के रंग को हाइलाइट करें _END,highlight border color,3,9
8,START_ हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। _END,the color and opacity of the highlight border,8,10
9,START_ भराई के रंग को हाइलाइट करें _END,highlight fill color,3,8


In [36]:
from sklearn.model_selection import train_test_split
X, y = lines['english'], lines['hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape


((23712,), (5928,))

In [37]:
X_train

24167    merged s                            
7189     task manager                        
2100     move a onto the jack of diamonds    
15131    run continue                        
11560    force                               
         ...                                 
21826    field s is mandatory please enter it
5443     six of clubs                        
865      select all                          
15993    new project creation has failed     
23930    quick open                          
Name: english, Length: 23712, dtype: object

In [38]:
encoder_input_data = np.zeros((2, max_length_src),dtype='float32')
decoder_input_data = np.zeros((2, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((2, max_length_tar, num_decoder_tokens),dtype='float32')

In [39]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [40]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [41]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [42]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [43]:
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 100

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 300)            806700    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 300)            1032000   ['input_2[0][0]']             
                                                                                              

In [44]:
model.save('eng-to-hindi.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
a, b = next(generate_batch())

In [46]:
b

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [47]:
X_train[4]

'a list of plugins that are disabled by default'

In [48]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=100,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Epoch 1/100


/tmp/ipykernel_42/4289143106.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


370/370 [==============================] - 49s 104ms/step - loss: 4.9933 - accuracy: 0.2601 - val_loss: 4.1115 - val_accuracy: 0.3191
Epoch 2/100
370/370 [==============================] - 14s 37ms/step - loss: 3.4418 - accuracy: 0.4026 - val_loss: 2.7855 - val_accuracy: 0.4883
Epoch 3/100
370/370 [==============================] - 14s 37ms/step - loss: 2.1545 - accuracy: 0.5939 - val_loss: 1.7461 - val_accuracy: 0.6850
Epoch 4/100
370/370 [==============================] - 14s 37ms/step - loss: 1.2559 - accuracy: 0.7664 - val_loss: 1.1782 - val_accuracy: 0.8053
Epoch 5/100
370/370 [==============================] - 14s 37ms/step - loss: 0.7937 - accuracy: 0.8540 - val_loss: 0.9169 - val_accuracy: 0.8549
Epoch 6/100
370/370 [==============================] - 14s 37ms/step - loss: 0.5544 - accuracy: 0.8936 - val_loss: 0.7754 - val_accuracy: 0.8806
Epoch 7/100
370/370 [==============================] - 14s 37ms/step - loss: 0.4135 - accuracy: 0.9170 - val_loss: 0.7058 - val_accuracy: 0.8

In [49]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [50]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [51]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [52]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: merged s
Actual Hindi Translation:  मिलाया से 
Predicted Hindi Translation:  मिलाया से 


In [53]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 21ms/step
Input English sentence: task manager
Actual Hindi Translation:  फाइल का नाम ढूंढो 
Predicted Hindi Translation:  कार्य प्रबंधक 


In [54]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 20ms/step
Input English sentence: move a onto the jack of diamonds
Actual Hindi Translation:  a को एक ईंट का गुलाम पर ले जाएँ 
Predicted Hindi Translation:  a को एक ईंट का गुलाम पर ले जाएँ 
